<a href="https://colab.research.google.com/github/AlinaKhan111/Exact__Inference/blob/main/exact_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pgmpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.3 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination


In [ ]:
# Defining the Bayesian network structure
model = BayesianNetwork([('Accident', 'TrafficJam'),
                         ('Roadworks', 'TrafficJam'),
                         ('Weather', 'TrafficJam'),
                         ('TimeOfDay', 'TrafficJam')])


In [ ]:
# CPD for accidents (10% chance of an accident)
cpd_accident = TabularCPD(variable='Accident', variable_card=2, values=[[0.9], [0.1]])

# CPD for roadworks (15% chance of roadworks)
cpd_roadworks = TabularCPD(variable='Roadworks', variable_card=2, values=[[0.85], [0.15]])

# CPD for weather (20% chance of bad weather)
cpd_weather = TabularCPD(variable='Weather', variable_card=2, values=[[0.8], [0.2]])

# CPD for time of day (peak hours vs off-peak: 50% chance of peak hour)
cpd_time_of_day = TabularCPD(variable='TimeOfDay', variable_card=2, values=[[0.5], [0.5]])


In [ ]:
# Corrected CPD for TrafficJam
cpd_traffic_jam = TabularCPD(variable='TrafficJam', variable_card=2,
                             values=[[0.95, 0.9, 0.85, 0.8, 0.7, 0.65, 0.6, 0.5,
                                      0.8, 0.75, 0.7, 0.65, 0.55, 0.5, 0.4, 0.3],  # P(TrafficJam=False)
                                     [0.05, 0.1, 0.15, 0.2, 0.3, 0.35, 0.4, 0.5,
                                      0.2, 0.25, 0.3, 0.35, 0.45, 0.5, 0.6, 0.7]],  # P(TrafficJam=True)
                             evidence=['Accident', 'Roadworks', 'Weather', 'TimeOfDay'],
                             evidence_card=[2, 2, 2, 2])


In [ ]:
# Adding CPDs to the model
model.add_cpds(cpd_accident, cpd_roadworks, cpd_weather, cpd_time_of_day, cpd_traffic_jam)

# Checking if the model is valid
assert model.check_model()


In [ ]:
# Initialize the inference object
inference = VariableElimination(model)

# Performing inference given evidence: bad weather and peak hours
result_traffic_jam = inference.query(variables=['TrafficJam'], evidence={'Weather': 1, 'TimeOfDay': 1})

# Output the result
print(result_traffic_jam)


+---------------+-------------------+
| TrafficJam    |   phi(TrafficJam) |
+===============+===================+
| TrafficJam(0) |            0.7393 |
+---------------+-------------------+
| TrafficJam(1) |            0.2608 |
+---------------+-------------------+


In [ ]:
# Performing inference given evidence: accident and roadworks
result_traffic_jam_with_accident = inference.query(variables=['TrafficJam'], evidence={'Accident': 1, 'Roadworks': 1})

# Output the result
print(result_traffic_jam_with_accident)


+---------------+-------------------+
| TrafficJam    |   phi(TrafficJam) |
+===============+===================+
| TrafficJam(0) |            0.4900 |
+---------------+-------------------+
| TrafficJam(1) |            0.5100 |
+---------------+-------------------+
